In [2]:
import random
import itertools

import pandas

In [3]:
DATA_PATH: str = "../../../data"
DATASET_PATH: str = f"{DATA_PATH}/processed/twitter.english.dataset.enriched.csv"

In [5]:
data = (
    pandas.read_csv(DATASET_PATH, index_col=[0])
    .pipe(
        lambda _df: _df[_df["author_id_reply"].isin(
            (
                (_df.groupby("author_id_reply").size() > 3)
                .pipe(lambda _srs: _srs[_srs])
            ).index
        )]
        )
)
display(len(data))
data.head()

5468

,id_post,id_reply,author_id_post,author_id_reply,author_first_name_post,author_last_name_post,author_party_post,text_post,text_reply,topics_post,topics_reply
0,1684217285567807488,1684217285567807488,131546062,1088163738950295552,Rick,Scott,Republican Party,Democrats scream every day about limiting Amer...,Funny that Hunter is being held to account yet...,"Tax evasion, Gun rights, Hypocrisy","Hunter Biden, Trump family, Accountability"
1,1682471504880050176,1682471504880050176,131546062,1088163738950295552,Rick,Scott,Republican Party,If our economy is doing as well as @JoeBiden s...,You’re lying,"Economic performance, Inflation, Biden's policies","Deception,Lying,Accusation"
2,1682337371126112512,1682337371126112512,131546062,1088163738950295552,Rick,Scott,Republican Party,Bottom line: our military should be focused on...,Why don’t you call Senator Tubberville and tel...,"National Security, Military Preparedness, Supp...","Military, Promotions, Enemies"
3,1681994109094080512,1681994109094080512,131546062,1088163738950295552,Rick,Scott,Republican Party,I'm fighting for amendments in the FY 24 NDAA ...,OK folks let’s get your shot glasses out and w...,"Communist China threat, US military reform, Na...","Rick Scott, Desperation, Woke Culture"
4,1681424977076322304,1681424977076322304,131546062,1088163738950295552,Rick,Scott,Republican Party,Why is the White House acting like America's i...,Blah blah blah,"Inflation, Joe Biden, Taxation",There is no tweet provided. Please provide a t...


In [6]:
positive_samples = {
    author: [
        sample for sample in itertools.combinations(group["text_post"].tolist(), 3)
    ]
    for author, group in data.groupby("author_id_reply")
}
sum(len(group) for group in positive_samples.values())

3489767

In [7]:
negative_samples = {
    author: [
        (
            *random.choice(positive_samples[author])[:2],
            random.choice(
                positive_samples[
                    random.choice(list(set(positive_samples.keys() - set([author]))))
                ]
            )[2],
        )
        for _ in range(450)
    ]
    for author in set(positive_samples.keys())
}
sum(len(group) for group in negative_samples.values())

140850

In [8]:
dataset = pandas.concat(
    [
        pandas.DataFrame(
            [
                dict(
                    history_1=sample[0],
                    history_2=sample[1],
                    stimulus=sample[2],
                    action=label,
                )
                for group in sets
                for sample in group
            ]
        ).sample(100_000)
        for label, sets in [
            (True, positive_samples.values()),
            (False, negative_samples.values()),
        ]
    ]
)

dataset.to_csv(DATASET_PATH.replace("enriched", "decision"))
len(dataset)

200000